# Monash Automated Essay Scoring
_School of Information Technology_<br>
_Monash University Malaysia_<br>
(c) Copyright, Ian Tan 2020

### Import pandas, nltk and sci-kit learn libraries

In [3]:
import numpy as np
import pandas as pd
from collections import defaultdict

from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm #SVR is in SVM
from sklearn.metrics import accuracy_score, confusion_matrix

### Import the EASE functions, which is located in the ease folder.

In [4]:
import sys
sys.path.insert(1, 'ease')
import create
import grade 
import model_creator 
import predictor_extractor 
import predictor_set 
import util_functions
import essay_set
import feature_extractor

from essay_set import EssaySet
from feature_extractor import FeatureExtractor

### Read the AES dataset from the folder aes

In [5]:
train_set = pd.read_csv("asap-aes/training_set_rel3.tsv", sep='\t', encoding="latin-1")
#train_set.head()

In [6]:
train_set = train_set[train_set['essay_set'] == 2]  # filter for set 2
#train_set.head()

In [7]:
train_set = train_set.reset_index() # resets index
#train_set.head()

In [8]:
train_set['essay'] = [entry.lower() for entry in train_set['essay']] # lower case for all words in essay
#train_set.head()

In [9]:
essays = train_set['essay']
scores = train_set['domain1_score']

In [10]:
essays

0       certain materials being removed from libraries...
1       write a persuasive essay to a newspaper reflec...
2       do you think that libraries should remove cert...
3       in @date1's world, there are many things found...
4       in life you have the 'offensive things'. the l...
5       a lot of @caps3 today are censored because of ...
6       how @caps4 you feel if your favorite book was ...
7       do you think that you should be able to take c...
8       the idea of the removal of 'offensive' materia...
9       all authors write for a purpose, whether it be...
10      have you seen a magazine, book, movies, etc., ...
11      personally i don't think libraries, movie stor...
12      a library is a place you can go to read, write...
13      yes and no, some materials such as books, musi...
14      when talking about censorship in libraries, i ...
15      dear newspaper,     @caps1 would you feel if s...
16      a lot of people that are in school have probab...
17      do you

### Collate the essay prompts
This consist of one essay from each set

In [36]:
essay_prompts = []

for i in range(1,9):
    file = "prompts/set" + str(i) + ".txt"
    f = open(file, "r", encoding="latin-1") # there are some 0x9x characters, hence need to specify encoding
    essay_prompts.append(f.read())
    
def get_essay_prompt(essay_set):
    return essay_prompts[essay_set-1]

### Create the essay sets

In [11]:
e_set = EssaySet()

for i in range(len(essays)):
    e_set.add_essay(essays[i], scores[i])

KeyboardInterrupt: 

In [ ]:
f_extractor = FeatureExtractor()